# Eat Safe, Love

## Notebook Set Up

In [30]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import json
import pandas as pd

In [31]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [32]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [33]:
# review the collections in our database

print(db.list_collection_names)


<bound method Database.list_collection_names of Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food')>


In [44]:
# assign the collection to a variable
establishments = db['establishments']

# Print contents of the collection

pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'Location': {'Latitude': {'$toDouble': '$Location.Latitude'},
              'Longitude': {'$toDouble': '$Location.Longitude'}},
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('657684401a5d6760d499ab78'),
 'geocode': {'latitude': '51.083812', 'long

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [45]:
# Find the establishments with a hygiene score of 20

query = {"establishments.RatingValue": 20}


# Use count_documents to display the number of documents in the result

print(establishments.count_documents(query))
      


# Display the first document in the results using pprint

pprint(establishments.find_one(query))



0
None


In [36]:
# Convert the result to a Pandas DataFrame

value_df = pd.DataFrame(list(establishments.find(query)))


# Display the number of rows in the DataFrame

print(len(value_df))


# Display the first 10 rows of the DataFrame

print(value_df.head(10))


0
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [37]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.

query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}


# Use count_documents to display the number of documents in the result

print(establishments.count_documents(query))

# Display the first document in the results using pprint

pprint(establishments.find_one(query))


0
None


In [38]:
# Convert the result to a Pandas DataFrame

london_df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame

print(len(london_df))


# Display the first 10 rows of the DataFrame

print(london_df.head(10))





0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [39]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude =  51.501476
longitude =  -0.140634

query =  {"Latitude": {"$gte": latitude - degree_search,
                        "$lte": latitude + degree_search},
                          "Longitude": {"$gte": longitude - degree_search,
                                         "$lte": longitude + degree_search},
                                           "RatingValue": 5}
sort = [("RatingValue", -1)]

# Print the results

pprint(establishments.find_one(query, sort=sort))



None


In [40]:
# Convert result to Pandas DataFrame

london_df = pd.DataFrame(list(establishments.find(query, sort=sort)))




### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [41]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [
    {"$match": {"RatingValue": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

# Print the number of documents in the result

pprint(list(establishments.aggregate(pipeline)))


# Print the first 10 results

pprint(list(establishments.aggregate(pipeline))[:10])

[]
[]


In [42]:
# Convert the result to a Pandas DataFrame

zero_df = pd.DataFrame(list(establishments.aggregate(pipeline)))


# Display the number of rows in the DataFrame

print(len(zero_df))

# Display the first 10 rows of the DataFrame

print(zero_df.head(10))


0
Empty DataFrame
Columns: []
Index: []
